In [1]:
import camelot
import matplotlib.pyplot as plt
import pandas as pd
import re
import datetime
import PyPDF2

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
filename="A:/Github Repos/UPS-project/SamplePDFs/2304 Trips.pdf"

In [5]:
dflist=[]

In [4]:
print("Extracting Trips")
pdf=open(filename,'rb')
lastpage=PyPDF2.PdfFileReader(pdf).getNumPages()
startpage=1
pgrange=str(startpage)+'-'+str(lastpage)
pdf.close()
    
print("  Obtained the page range: ",pgrange)
print("  Reading pdf. Getting Trips this may take a few minutes...")
    
while True:
    try:
        tables=camelot.read_pdf(filename,pages=pgrange,flavor='stream',table_areas=['13.4,573,390,61','390,573,774,61'],columns=['47.5,67.8,136.1,168.9,196.6,214.7,231.2,249.2,265,288.7,317.5,353.2','443.4,464.2,531.4,565,594.9,613.1,629.1,644.5,663.2,686.2,713.4,749.1'],strip_text='--\n',split_text=True)
        break
    except:
        startpage=startpage+1
        pgrange=str(startpage)+'-'+str(lastpage)
    
print("  Done reading pdf")

Extracting Trips
  Obtained the page range:  1-151
  Reading pdf. Getting Trips this may take a few minutes...
  Done reading pdf


In [6]:
print("  Parsing Trips")
    
for n in range(0,len(tables)):
# initialize variables to keep track of the current and previous trip ids
    tripid_index=0
    prev_trip_id = None
    curr_trip_id = None
    # loop through each row of the dataframe
    for index, row in tables[n].df.iterrows():
        # check if the current row contains a trip id
    
        if 'Trip Id:' in row[0]:
            # if it does, split the dataframe into two parts
            if prev_trip_id is not None:
                dflist.append((tables[n].df.loc[tripid_index:index-1]).reset_index(drop=True))
                tables[n].df = tables[n].df.loc[index:]
            
            tripid_index=index
            curr_trip_id = row[0]
            prev_trip_id = curr_trip_id
if tripid_index==0:
    del tables[n].df
else:
    # append the last part of the dataframe to the result list    
    dflist.append(tables[n].df.reset_index(drop=True))
    
print("  Finished Parsing trips")

  Parsing Trips
  Finished Parsing trips


Creating the trips list

In [7]:
trips_list=[]

In [8]:
df=pd.DataFrame(columns=["Trip Id", "DH", "Start time", "End time","Layovers","TAFB","Dest"])

In [9]:
trips_list.append(df)

In [15]:
trips_list[0]

,Trip Id,DH,Start time,End time,Layovers,TAFB,Dest
0,1,NaN,NaN,NaN,NaN,7.95,NaN


Adding the Trip ID

In [11]:
trips_list[0].loc[0,"Trip Id"]=int(re.findall('\d+',dflist[0].iloc[0,0])[0])

Adding TAFB

In [14]:
hours, minutes=dflist[0].iloc[9,12].split("h")
trips_list[0].loc[0,"TAFB"]=int(hours)+(int(minutes)/60)

Adding number of DH or CM

In [20]:
DH=0
if re.findall( r'\bDH\b',dflist[0].iloc[5,1]) or re.findall( r'\bCM\b',dflist[0].iloc[5,1]):
    DH=DH+1

for i in range(len(dflist[0])-1,-1,-1):
    string=dflist[0].iloc[i,1]
    if string!='':
        if re.findall( r'\bDH\b',string) or re.findall( r'\bCM\b',string):
            DH=DH+1
            break
trips_list[0].loc[0,"DH"]=DH

0
